## Clone the GitHub Repo

In [1]:
!git clone https://github.com/bhargav-borah/CIFAR10ImageClassification

fatal: destination path 'CIFAR10ImageClassification' already exists and is not an empty directory.


## Import the dependencies

In [2]:
import torch
from torch import nn
from torchvision.transforms import transforms

from pathlib import Path
import os
from CIFAR10ImageClassification.modules.accuracy import accuracy_fn

## Set up device-agnostic code

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Create train and test dataloaders from CIFAR-10 dataset

In [4]:
os.getcwd()

'/kaggle/working'

In [5]:
os.chdir('CIFAR10ImageClassification/modules')

In [6]:
os.getcwd()

'/kaggle/working/CIFAR10ImageClassification/modules'

In [7]:
from data_setup import get_cifar10_dataloaders

In [8]:
data_dir = 'data'

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

In [10]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader, test_dataloader, class_names = get_cifar10_dataloaders(data_dir=data_dir,
                                                                        transform=transform,
                                                                        batch_size=BATCH_SIZE,
                                                                        num_workers=NUM_WORKERS)

Files already downloaded and verified
Files already downloaded and verified


## Create a model

In [11]:
class CIFAR10ModelV6(nn.Module):
    def __init__(self, input_shape: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=8,
                      kernel_size=2,
                      stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=8,
                      out_channels=8,
                      kernel_size=2,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=1)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=8,
                      out_channels=8,
                      kernel_size=2,
                      stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=8,
                      out_channels=8,
                      kernel_size=2,
                      stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=1)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=380192,
                      out_features=len(class_names))
        )
    
    def forward(self, X: torch.Tensor):
        return self.classifier(self.block_2(self.block_1(X)))

In [12]:
8 * 218 * 218

380192

In [13]:
model_v6_1 = CIFAR10ModelV6(input_shape=3,
                            output_shape=len(class_names))
model_v6_1

CIFAR10ModelV6(
  (block_1): Sequential(
    (0): Conv2d(3, 8, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(2, 2), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(8, 8, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(2, 2), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=380192, out_features=10, bias=True)
  )
)

## Train the model

In [14]:
from model_builder import fit

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_v6_1.parameters(),
                             lr=1e-3)

history_v6_1 = fit(model=model_v6_1,
                   epochs=10,
                   train_dataloader=train_dataloader,
                   test_dataloader=test_dataloader,
                   loss_fn=loss_fn,
                   optimizer=optimizer,
                   device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Training loss:  1.8327726125717163 | Training accuracy: 0.34936821460723877 | Test loss: 1.7020879983901978 | Test accuracy: 0.4048522412776947
Training loss:  1.726481318473816 | Training accuracy: 0.3943138122558594 | Test loss: 1.8245716094970703 | Test accuracy: 0.34814295172691345
Training loss:  1.6645562648773193 | Training accuracy: 0.41442736983299255 | Test loss: 1.626421570777893 | Test accuracy: 0.421026349067688
Training loss:  1.5286686420440674 | Training accuracy: 0.46111243963241577 | Test loss: 1.5161957740783691 | Test accuracy: 0.4553714096546173
Training loss:  1.3685308694839478 | Training accuracy: 0.5201135873794556 | Test loss: 1.4074598550796509 | Test accuracy: 0.4991014301776886
Training loss:  1.2825416326522827 | Training accuracy: 0.5506637692451477 | Test loss: 1.3900295495986938 | Test accuracy: 0.513877809047699
Training loss:  1.209240436553955 | Training accuracy: 0.5770553350448608 | Test loss: 1.3703296184539795 | Test accuracy: 0.5142771601676941
